# Building custom image for function

In this tutorial we will describe how to build custom docker image for function.

In this tutorial we will be following 3 steps to deploy our function with custom docker image:
1. implement function template 
2. define dockerfile, build it and push to registry
3. upload

First we will implement function entrypoint by following template. All functions with custom docker images must follow same template structure. 

We need to create class `Runner` and implement `run` method that will be called during invocation of the function and results of the run method will be returned as result of the function.

Let's create [runner.py](./custom_function/runner.py) file with following content
```python
class Runner:
    def run(self, arguments: dict) -> dict:
        # this is just an example
        # your function can call for other modules, function, etc.
        return {
            **arguments,
            **{
                "answer": 42
            }
        }
```

As a next step let's define and build our custom docker image.

Dockerfile will be extending base serverless node image and adding required packages and structure to it. 

In our simple case it will look something like this

```Dockerfile
FROM icr.io/quantum-public/quantum-serverless-ray-node:0.9.0-py39

# install all necessary dependencies for your custom image

# copy our function implementation in `/runner.py` of the docker image
COPY ./runner.py ./runner.py
...
```

and then we need to build it

In [10]:
! docker build -t icr.io/quantum-public/my-custom-function-image:v4 ./custom_function

In [ ]:
! docker push icr.io/quantum-public/my-custom-function-image:v4

We got to our final step of function development - uploading to serverless.

Let define `QiskitFunction` with image we just build, give it a name and upload it.

In [1]:
import os
from quantum_serverless import QiskitFunction, ServerlessClient

serverless = ServerlessClient(
    token=os.environ.get("GATEWAY_TOKEN", "awesome_token"),
    host=os.environ.get("GATEWAY_HOST", "http://localhost:8010"),
)
serverless

<gateway-client>

In [2]:
function_with_custom_image = QiskitFunction(
    title="custom-image-function",
    image="icr.io/quantum-public/my-custom-function-image:v4"
)
function_with_custom_image

QiskitFunction(custom-image-function)

In [3]:
serverless.upload(function_with_custom_image)

'custom-image-function'

Now it's time to test our newly uploaded function.

In [4]:
functions = {f.title: f for f in serverless.list()}
my_function = functions.get("custom-image-function")
my_function

QiskitFunction(custom-image-function)

In [5]:
job = my_function.run(test_argument_one=1, test_argument_two="two")
job

<Job | dc6d3a90-bead-4f12-b469-21c7ac9c7d16>

In [6]:
job.result()

{'test_argument_one': 1, 'test_argument_two': 'two', 'answer': 42}